In [56]:
import pandas as pd
import random

historia:
loty startuja z ziemi - poczatek ukladu wspolrzednych.
im planeta w galaktyce dalej od ziemii tym:
- wiecej pracownikow
- drozszy lot
- mozliwe ze bedzie dostepny tylko w jedna strone

Im:
- statek ma wieksza capacity (wielkosc),
-
tym cena wyzsza

planety w jednej galaktyce maja zblizone wspolrzedne.


jak ktos pojechal tylko w jedna strone - to juz nie moze pojawic sie w transakcjach i bookingach.

niech salary pracownikow bedzie najwyzsze dla commander a najnizsze dla technitian i neich zalezy od doswiadczenia w firmie.



planets + coordinates + galaxy

In [57]:
import pandas as pd
import random

# Galaxies and their base coordinates (for clustering)
galaxies = [
    "Andros Spiral", "Helion Prime", "Novara Cluster", "Vortexis", "Celestara",
    "Omega Verge", "Halcyon Drift", "Nemera Arc", "Quasari Belt", "Eclipse Halo",
    "Astralis", "Kronis Ring", "Auroria", "Zerion Reach", "Polaris Nexus"
]

# Step 1: Generate galaxy table
galaxy_data = []
galaxy_coords = {}

for i, name in enumerate(galaxies, start=1):
    galaxy_data.append({
        "galaxy_id": i,
        "galaxy": name
    })
    # base coordinates for clustered planets
    galaxy_coords[i] = {
        "base_x": random.uniform(-1000, 1000),
        "base_y": random.uniform(-1000, 1000),
        "base_z": random.uniform(-1000, 1000)
    }

# Step 2: Generate planets clustered by galaxy
planet_names = [
    "Aetheria", "Zorath", "Nyxos", "Virella", "Targus", "Ecliptor", "Orionis", "Caelora",
    "Draconis", "Pyrrhos", "Talmera", "Lunaris", "Zephyra", "Solvana", "Obscura",
    "Cryon", "Valtor", "Serenos", "Arctyra", "Phobaris"
]

planet_data = []
for i, name in enumerate(planet_names):
    galaxy_id = random.randint(1, len(galaxies))
    base = galaxy_coords[galaxy_id]
    planet_data.append({
        "planet_id": i + 1,
        "planet": name,
        "x_coord": round(base["base_x"] + random.uniform(-100, 100), 2),
        "y_coord": round(base["base_y"] + random.uniform(-100, 100), 2),
        "z_coord": round(base["base_z"] + random.uniform(-100, 100), 2),
        "galaxy_id": galaxy_id
    })

# Convert to DataFrames
df_galaxies = pd.DataFrame(galaxy_data)
df_planets = pd.DataFrame(planet_data)

# Save to CSV
# df_galaxies.to_csv("galaxies_from_diagram.csv", index=False)
# df_planets.to_csv("planets_from_diagram.csv", index=False)

# # Show preview
# print("Preview of generated planet data:")
# print(df_planets.head())


In [58]:
df_planets

,planet_id,planet,x_coord,y_coord,z_coord,galaxy_id
0,1,Aetheria,-88.30,667.69,903.12,11
1,2,Zorath,27.94,487.40,-810.56,5
2,3,Nyxos,332.93,267.44,-80.21,10
3,4,Virella,476.18,112.86,-739.32,6
4,5,Targus,-786.07,914.14,-268.14,1
5,6,Ecliptor,797.86,-353.22,170.29,8
6,7,Orionis,-41.57,629.90,877.46,11
7,8,Caelora,864.61,-243.78,142.11,3
8,9,Draconis,471.07,-567.64,864.37,15
9,10,Pyrrhos,-63.93,503.83,-826.97,5


trip types

In [ ]:
import math
import pandas as pd

# Zakładam, że df_planets już istnieje (jeśli nie, trzeba go wcześniej załadować z CSV)

# Step 1: Generate a trip type for each planet
trip_types = []

for i, row in df_planets.iterrows():
    distance_from_origin = math.sqrt(row["x_coord"]**2 + row["y_coord"]**2 + row["z_coord"]**2)
    is_round_trip = distance_from_origin < 900  # one-way if far from (0,0,0)
    
    trip_types.append({
        "triptype_id": i + 1,
        "name": f"Mission to {row['planet']}",
        #"coordinates": f"X:{row['x_coord']}, Y:{row['y_coord']}, Z:{row['z_coord']}",
        "description": f"Explore the unique surface and atmosphere of planet {row['planet']}.",
        "is_round_trip": is_round_trip,
        "planet_id": row["planet_id"]
    })

# Convert to DataFrame and save
df_trip_types = pd.DataFrame(trip_types)

# Zmień ścieżkę na swoją lokalną, np. na Pulpit
#df_trip_types.to_csv("trip_types_linked_to_planets.csv", index=False)

# Show previe


In [61]:
df_trip_types

,triptype_id,name,description,is_round_trip,planet_id
0,1,Mission to Aetheria,Explore the unique surface and atmosphere of p...,False,1
1,2,Mission to Zorath,Explore the unique surface and atmosphere of p...,False,2
2,3,Mission to Nyxos,Explore the unique surface and atmosphere of p...,True,3
3,4,Mission to Virella,Explore the unique surface and atmosphere of p...,True,4
4,5,Mission to Targus,Explore the unique surface and atmosphere of p...,False,5
5,6,Mission to Ecliptor,Explore the unique surface and atmosphere of p...,True,6
6,7,Mission to Orionis,Explore the unique surface and atmosphere of p...,False,7
7,8,Mission to Caelora,Explore the unique surface and atmosphere of p...,False,8
8,9,Mission to Draconis,Explore the unique surface and atmosphere of p...,False,9
9,10,Mission to Pyrrhos,Explore the unique surface and atmosphere of p...,False,10


employees

In [68]:
import random
import pandas as pd
from datetime import datetime, timedelta
from faker import Faker

fake = Faker()

positions = ["Technician", "Mission Specialist", "Commander", "Pilot", "Navigator"]
position_weights = [1, 1, 1, 3, 3]

salary_ranges = {
    "Technician": (4000, 6000),
    "Mission Specialist": (8000, 11000),
    "Commander": (9000, 12000),
    "Pilot": (6000, 8500),
    "Navigator": (5500, 8000)
}

# Parametry czasu
start_date = datetime(2023, 1, 1)
end_random_hire = datetime(2025, 6, 1)
today = datetime.today()

employees_data = []
num_employees = 30
pct_early_hires = 0.5
early_hires = int(num_employees * pct_early_hires)

for i in range(num_employees):
    first_name = fake.first_name()
    last_name = fake.last_name()

    position = random.choices(positions, weights=position_weights, k=1)[0]
    low, high = salary_ranges[position]

    # Hire date
    if i < early_hires:
        hire_date = start_date
    else:
        rand_days = random.randint(0, (end_random_hire - start_date).days)
        hire_date = start_date + timedelta(days=rand_days)

    # Staż w latach (pełne lata)
    years_worked = max(0, (today - hire_date).days // 365)

    # Wynagrodzenie bazowe + podwyżki
    base_salary = random.randrange(low, high + 1, 100)
    adjusted_salary = round(base_salary * (1.07 ** years_worked), -2)  # zaokrąglenie do setek

    employees_data.append({
        "employee_id": i + 1,
        "first_name": first_name,
        "last_name": last_name,
        "position": position,
        "hire_date": hire_date.strftime("%Y-%m-%d"),
        "salary": adjusted_salary
    })

# DataFrame
df_employees = pd.DataFrame(employees_data)



In [69]:
df_employees.to_csv("employees.csv", index=False)
df_employees

,employee_id,first_name,last_name,position,hire_date,salary
0,1,Adam,Allen,Navigator,2023-01-01,8400.0
1,2,Victoria,Hansen,Pilot,2023-01-01,7200.0
2,3,Michele,Green,Pilot,2023-01-01,7300.0
3,4,Charles,Acosta,Navigator,2023-01-01,8900.0
4,5,Mark,Nunez,Pilot,2023-01-01,8900.0
5,6,Jessica,Watkins,Technician,2023-01-01,5200.0
6,7,Zachary,Mitchell,Pilot,2023-01-01,8900.0
7,8,Sheri,Ramos,Technician,2023-01-01,4700.0
8,9,Robert,Harrison,Pilot,2023-01-01,8400.0
9,10,Colleen,Moore,Technician,2023-01-01,5600.0


trips+rocket

In [24]:
def is_employee_available(emp_id, new_start, new_end):
    for trip in trip_employees:
        if trip["employee_id"] == emp_id:
            # znajdź zakres dat tej misji
            trip_id = trip["trip_id"]
            trip_info = next((t for t in trips if t["trip_id"] == trip_id), None)
            if trip_info:
                existing_start = datetime.strptime(trip_info["departure_date"], "%Y-%m-%d")
                existing_end = datetime.strptime(trip_info["return_date"], "%Y-%m-%d")
                # sprawdź konflikt
                if not (new_end < existing_start or new_start > existing_end):
                    return False
    return True


In [25]:
# Update employee and trip generation to ensure each trip has a Pilot and Navigator
from datetime import datetime
from datetime import timedelta

rockets=['Stellar Vortex','Nebula Strider','Quantum Lance','Solaris Ark']
employees=["Technician", "Mission Specialist", "Commander","Pilot", "Navigator"]
start_date=datetime(2023,1,1)
trips = []
trip_employees = []

for i, row in df_trip_types.iterrows():
    num_variants = random.randint(1, 3)  # 1-3 trips per trip type
    for _ in range(num_variants):
        trip_id = len(trips) + 1
        departure = start_date + timedelta(days=random.randint(0,600))
        duration = random.randint(5, 30)
        return_date = departure + timedelta(days=duration)
        rocket_id = random.randint(1, len(rockets))

        trips.append({
            "trip_id": trip_id,
            "triptype_id": row["triptype_id"],
            "rocket_id": rocket_id,
            "departure_date": departure.strftime("%Y-%m-%d"),
            "return_date": return_date.strftime("%Y-%m-%d"),
            "status": random.choice(["Scheduled", "Completed", "Delayed"])
        })

        is_round_trip = row["is_round_trip"]
        min_roles = ["Pilot", "Navigator"]
        other_roles = ["Technician", "Mission Specialist", "Commander"]
        n_optional = random.randint(1, 2) if is_round_trip else 0  # more if round trip

        assigned = []
        # Add required roles
        # Dodaj obowiązkowego Pilota i Nawigatora
        for role in min_roles:
            available = [e for e in employees_data if e["position"] == role and e["employee_id"] not in assigned and is_employee_available(e["employee_id"], departure, return_date)]
        if not available:
            raise ValueError(f"No available employee for role: {role} (without conflict)")
    
        emp = random.choice(available)
        assigned.append(emp["employee_id"])
        trip_employees.append({
        "trip_id": trip_id,
        "employee_id": emp["employee_id"],
        "role_in_trip": emp["position"]
    })


        # Add optional roles
        for _ in range(n_optional):
            available = [e for e in employees_data if e["position"] in other_roles and e["employee_id"] not in assigned]
            if available:
                emp = random.choice(available)
                assigned.append(emp["employee_id"])
                trip_employees.append({
                    "trip_id": trip_id,
                    "employee_id": emp["employee_id"],
                    "role_in_trip": emp["position"]
                })

# Update DataFrames and save
df_trips = pd.DataFrame(trips)
df_trip_employees = pd.DataFrame(trip_employees)
df_employees = pd.DataFrame(employees)
rocket_data = [
    {"rocket_id": 1, "name": "Stellar Vortex", "capacity": 6},
    {"rocket_id": 2, "name": "Nebula Strider", "capacity": 10},
    {"rocket_id": 3, "name": "Quantum Lance", "capacity": 4},
    {"rocket_id": 4, "name": "Solaris Ark", "capacity": 8},
]

df_rockets = pd.DataFrame(rocket_data)

df_trips.to_csv("trips_generated.csv", index=False)
df_trip_employees.to_csv("trip_employees.csv", index=False)
df_employees.to_csv("employees_generated.csv", index=False)
df_rockets.to_csv("rockets_generated.csv", index=False)

# Preview assigned trip employees
#import ace_tools as tools; tools.display_dataframe_to_user(name="Trip Employees with Required Roles", dataframe=df_trip_employees)


In [26]:
df_trip_employees

,trip_id,employee_id,role_in_trip
0,1,3,Navigator
1,2,5,Navigator
2,3,3,Navigator
3,4,18,Navigator
4,5,18,Navigator
...,...,...,...
64,40,13,Mission Specialist
65,41,19,Navigator
66,41,27,Commander
67,42,19,Navigator


najbardziej sie spoznia - quantum lance

In [27]:
from datetime import datetime, timedelta
import random
import pandas as pd

# Dane bazowe
rockets = ['Stellar Vortex', 'Nebula Strider', 'Quantum Lance', 'Solaris Ark']
employees_data = [
    {"employee_id": i + 1, "position": pos}
    for i, pos in enumerate(["Technician", "Mission Specialist", "Commander", "Pilot", "Navigator"] * 10)
]
start_date = datetime(2023, 1, 1)
cutoff_date = datetime(2025, 6, 1)
delayed_rocket = "Quantum Lance"

# Przygotowanie
trips = []
trip_employees = []

for i, row in df_trip_types.iterrows():
    num_variants = random.randint(1, 3)
    for _ in range(num_variants):
        trip_id = len(trips) + 1
        departure = start_date + timedelta(days=random.randint(0, 1500))
        duration = random.randint(5, 30)
        return_date = departure + timedelta(days=duration)
        rocket_id = random.randint(1, len(rockets))
        rocket_name = rockets[rocket_id - 1]

        # Status logic
        if return_date < cutoff_date:
            status = "Completed"
        elif rocket_name == delayed_rocket:
            status = "Delayed"
        else:
            status = random.choice(["Scheduled", "Delayed"])

        trips.append({
            "trip_id": trip_id,
            "triptype_id": row["triptype_id"],
            "rocket_id": rocket_id,
            "departure_date": departure.strftime("%Y-%m-%d"),
            "return_date": return_date.strftime("%Y-%m-%d"),
            "status": status
        })

        is_round_trip = row["is_round_trip"]
        min_roles = ["Pilot", "Navigator"]
        other_roles = ["Technician", "Mission Specialist", "Commander"]
        n_optional = random.randint(1, 2) if is_round_trip else 0

        assigned = []# Dodaj obowiązkowego Pilota i Nawigatora
        for role in ["Pilot", "Navigator"]:
            available = [
        e for e in employees_data
        if e["position"] == role
        and e["employee_id"] not in assigned
        and is_employee_available(e["employee_id"], departure, return_date)
    ]
    
            if not available:
                raise ValueError(f"No available employee for role: {role} (without conflict)")
    
            emp = random.choice(available)
            assigned.append(emp["employee_id"])
            trip_employees.append({
        "trip_id": trip_id,
        "employee_id": emp["employee_id"],
        "role_in_trip": emp["position"]
    })

        

        for _ in range(n_optional):
            available = [e for e in employees_data if e["position"] in other_roles and e["employee_id"] not in assigned]
            if available:
                emp = random.choice(available)
                assigned.append(emp["employee_id"])
                trip_employees.append({
                    "trip_id": trip_id,
                    "employee_id": emp["employee_id"],
                    "role_in_trip": emp["position"]
                })

# DataFrames
df_trips = pd.DataFrame(trips)
df_trip_employees = pd.DataFrame(trip_employees)
df_employees = pd.DataFrame(employees_data)
rocket_data = [
    {"rocket_id": 1, "name": "Stellar Vortex", "capacity": 6},
    {"rocket_id": 2, "name": "Nebula Strider", "capacity": 10},
    {"rocket_id": 3, "name": "Quantum Lance", "capacity": 4},
    {"rocket_id": 4, "name": "Solaris Ark", "capacity": 8},
]

df_rockets = pd.DataFrame(rocket_data)

# # Save
# df_trips.to_csv("trips_generated.csv", index=False)
# df_trip_employees.to_csv("trip_employees.csv", index=False)
# df_employees.to_csv("employees_generated.csv", index=False)
# df_rockets.to_csv("rockets_generated.csv", index=False)


In [28]:
df_trips

,trip_id,triptype_id,rocket_id,departure_date,return_date,status
0,1,1,1,2024-06-17,2024-06-25,Completed
1,2,1,1,2025-05-28,2025-06-25,Delayed
2,3,2,2,2023-06-21,2023-07-11,Completed
3,4,2,4,2026-10-23,2026-11-13,Scheduled
4,5,3,1,2025-09-06,2025-09-19,Delayed
5,6,4,2,2024-04-16,2024-04-30,Completed
6,7,5,2,2023-10-03,2023-10-08,Completed
7,8,6,4,2026-07-12,2026-07-19,Scheduled
8,9,7,2,2024-10-31,2024-11-09,Completed
9,10,8,1,2024-02-13,2024-03-06,Completed


transaction + booking

In [36]:
import math

# Step 1: Generate clients
from faker import Faker
fake = Faker()
clients = [{
    "client_id": i + 1,
    "first_name": fake.first_name(),
    "last_name": fake.last_name(),
    "email": fake.email(),
    "phone": fake.phone_number()
} for i in range(50)]

df_clients = pd.DataFrame(clients)

# Step 2: Generate bookings (1-3 per client, random trips)
bookings = []
for client in clients:
    n_bookings = random.randint(1, 3)
    booked_trip_ids = random.sample(df_trips["trip_id"].tolist(), n_bookings)
    for trip_id in booked_trip_ids:
        bookings.append({
            "booking_id": len(bookings) + 1,
            "client_id": client["client_id"],
            "trip_id": trip_id,
            "seat_number": random.randint(1, 12)
        })

df_bookings = pd.DataFrame(bookings)


# Planet coordinates (from triptype -> planet)
planet_coords = df_trip_types.set_index("triptype_id")["coordinates"].to_dict()
round_trip_flags = df_trip_types.set_index("triptype_id")["is_round_trip"].to_dict()
triptypes_planet = df_trip_types.set_index("triptype_id")["planet_id"].to_dict()
rockets_capacity = df_rockets.set_index("rocket_id")["capacity"].to_dict()

# Count number of bookings per trip
#booking_counts = df_bookings.groupby("trip_id").size().to_dict()
# Liczba miejsc zarezerwowanych przez danego klienta na konkretny lot
booking_seat_counts = (
    df_bookings.groupby(["client_id", "trip_id"]).size().to_dict()
)

transactions = []
for booking in bookings:
    trip_id = booking["trip_id"]
    trip_row = df_trips[df_trips["trip_id"] == trip_id].iloc[0]
    triptype_id = trip_row["triptype_id"]
    rocket_id = trip_row["rocket_id"]
    coords = planet_coords[triptype_id]

    # Calculate distance
    coord_parts = [float(val.split(":")[1]) for val in coords.split(", ")]
    distance = math.sqrt(sum(c ** 2 for c in coord_parts))

    # Get employee count for trip
    employee_count = df_trip_employees[df_trip_employees["trip_id"] == trip_id].shape[0]

    base_price = 10000
    base_price += booking_seat_counts[trip_id] * 200           # more passengers
    base_price += (distance / 10)                         # distance
    base_price += rockets_capacity[rocket_id] * 100       # rocket size
    base_price += employee_count * 150                    # staffing cost
    if round_trip_flags[triptype_id]:
        base_price *= 1.25                                # round trip multiplier

    key = (booking["client_id"], trip_id)

# tylko jeśli ta transakcja jeszcze nie istnieje (1 transakcja = 1 klient x trip)
    if key not in {(t["client_id"], t["trip_id"]) for t in transactions}:
        num_seats = booking_seat_counts.get(key, 1)
        amount = round(base_price * num_seats, 2)  # cena zależna od liczby miejsc

        transactions.append({
        "transaction_id": len(transactions) + 1,
        "client_id": booking["client_id"],
        "trip_id": trip_id,
        "num_seats": num_seats,
        "amount": amount,
        "payment_date": fake.date_between(start_date='-1y', end_date='today').strftime("%Y-%m-%d"),
        "method": random.choice(["card", "transfer", "crypto"])
    })



df_transactions = pd.DataFrame(transactions)

# Save all new CSVs
# df_clients.to_csv("clients_generated.csv", index=False)
# df_bookings.to_csv("bookings_generated.csv", index=False)
# df_transactions.to_csv("transactions_generated.csv", index=False)


KeyError: 22

In [ ]:
import math
import random
import pandas as pd
from faker import Faker
from datetime import datetime

fake = Faker()

# Step 1: Generate clients
clients = []

for i in range(50):
    first = fake.first_name()
    last = fake.last_name()
    email_user = f"{first}.{last}".lower().replace(" ", "")
    domain = fake.free_email_domain()
    email = f"{email_user}@{domain}"
    
    clients.append({
        "client_id": i + 1,
        "first_name": first,
        "last_name": last,
        "email": email,
        "phone": fake.phone_number()
    })


df_clients = pd.DataFrame(clients)

# Step 2: Generate bookings (1–3 per client, max rocket capacity, seats obok siebie)
trip_capacity = df_rockets.set_index("rocket_id")["capacity"].to_dict()
trip_booked_seats = {trip_id: set() for trip_id in df_trips["trip_id"]}

bookings = []

for client in clients:
    n_reservations = random.randint(1, 3)
    available_trip_ids = df_trips["trip_id"].tolist()
    random.shuffle(available_trip_ids)
    chosen_trips = available_trip_ids[:n_reservations]

    for trip_id in chosen_trips:
        rocket_id = df_trips.loc[df_trips["trip_id"] == trip_id, "rocket_id"].values[0]
        capacity = trip_capacity[rocket_id]
        booked_seats = trip_booked_seats[trip_id]

        requested_seats = random.randint(1, 3)
        available_seats = sorted(set(range(1, capacity + 1)) - booked_seats)

        # Znajdź sąsiednie miejsca
        seat_block = None
        for i in range(len(available_seats) - requested_seats + 1):
            block = available_seats[i:i + requested_seats]
            if block[-1] - block[0] == requested_seats - 1:
                seat_block = block
                break

        if seat_block:
            for seat in seat_block:
                bookings.append({
                    "booking_id": len(bookings) + 1,
                    "client_id": client["client_id"],
                    "trip_id": trip_id,
                    "seat_number": seat
                })
                trip_booked_seats[trip_id].add(seat)

df_bookings = pd.DataFrame(bookings)

# Step 3: Generate transactions
planet_coords = df_trip_types.set_index("triptype_id")["coordinates"].to_dict()
round_trip_flags = df_trip_types.set_index("triptype_id")["is_round_trip"].to_dict()
rockets_capacity = df_rockets.set_index("rocket_id")["capacity"].to_dict()

# Seats per (client_id, trip_id)
booking_seat_counts = df_bookings.groupby(["client_id", "trip_id"]).size().to_dict()

transactions = []
existing_keys = set()

for booking in bookings:
    trip_id = booking["trip_id"]
    client_id = booking["client_id"]
    key = (client_id, trip_id)

    if key in existing_keys:
        continue
    existing_keys.add(key)

    trip_row = df_trips[df_trips["trip_id"] == trip_id].iloc[0]
    triptype_id = trip_row["triptype_id"]
    rocket_id = trip_row["rocket_id"]
    coords = planet_coords[triptype_id]
    
    # Parse coordinates
    coord_parts = [float(val.split(":")[1]) for val in coords.split(", ")]
    distance = math.sqrt(sum(c ** 2 for c in coord_parts))
    
    employee_count = df_trip_employees[df_trip_employees["trip_id"] == trip_id].shape[0]
    num_seats = booking_seat_counts.get(key, 1)

    base_price = 50000
    base_price += num_seats * 10000
    base_price += (distance / 10)
    base_price += rockets_capacity[rocket_id] * 1000
    base_price += employee_count * 15000
    if round_trip_flags[triptype_id]:
        base_price *= 1.25

    amount = round(base_price, 2)

    transactions.append({
        "transaction_id": len(transactions) + 1,
        "client_id": client_id,
        "trip_id": trip_id,
        "num_seats": num_seats,
        "amount": amount,
        "payment_date": fake.date_between(start_date='-1y', end_date='today').strftime("%Y-%m-%d"),
        "method": random.choice(["card", "transfer", "crypto"])
    })

df_transactions = pd.DataFrame(transactions)



In [43]:
df_transactions

,transaction_id,client_id,trip_id,num_seats,amount,payment_date,method
0,1,1,21,3,14575.64,2024-09-04,transfer
1,2,2,33,3,15124.85,2025-01-21,card
2,3,2,16,2,11218.57,2025-03-14,card
3,4,2,3,1,11615.58,2024-12-25,crypto
4,5,3,14,2,11193.46,2025-05-02,crypto
...,...,...,...,...,...,...,...
61,62,43,32,1,14075.76,2024-10-04,transfer
62,63,43,23,1,11015.21,2025-03-05,card
63,64,45,31,2,14138.26,2024-10-29,crypto
64,65,46,29,3,12002.07,2025-03-04,transfer


In [39]:
df_bookings

,booking_id,client_id,trip_id,seat_number
0,1,1,4,1
1,2,1,18,1
2,3,2,21,1
3,4,2,23,1
4,5,2,23,2
...,...,...,...,...
136,137,45,7,10
137,138,46,11,10
138,139,47,3,8
139,140,47,3,9


In [59]:
df_clients

,client_id,first_name,last_name,email,phone
0,1,Aaron,Thomas,fjohnson@example.org,001-398-722-2960x3955
1,2,Jessica,Guerrero,heatherbarnett@example.org,891.580.1808x790
2,3,Kaylee,Henry,roger39@example.com,616.770.2677x9489
3,4,David,Arias,janice29@example.net,(993)876-0202x4382
4,5,Christopher,Lopez,fordsamantha@example.net,370.986.1467x246
5,6,Jose,Hardin,michael31@example.net,001-244-513-6668x69316
6,7,Austin,Castillo,paul44@example.org,533-847-3419
7,8,Jason,Fisher,xbowen@example.org,+1-907-761-3202x0700
8,9,Christine,Hill,codyball@example.com,(628)233-1108
9,10,Jason,Bishop,meltonjoseph@example.org,659-258-5090


cost

In [44]:

# Count employees per trip
employee_counts = df_trip_employees.groupby("trip_id").size().to_dict()
# Sum of transactions per trip
income_per_trip = df_transactions.groupby("trip_id")["amount"].sum().to_dict()
# Triptype info
planet_coords = df_trip_types.set_index("triptype_id")["coordinates"].to_dict()
round_trip_flags = df_trip_types.set_index("triptype_id")["is_round_trip"].to_dict()

costs = []
for trip in df_trips.itertuples():
    trip_id = trip.trip_id
    triptype_id = trip.triptype_id
    coords = planet_coords[triptype_id]
    coord_parts = [float(val.split(":")[1]) for val in coords.split(", ")]
    distance = math.sqrt(sum(c ** 2 for c in coord_parts))
    is_round = round_trip_flags[triptype_id]
    n_employees = employee_counts.get(trip_id, 1)
    max_total_cost = income_per_trip.get(trip_id, 10000)

    # Base cost estimate
    base_cost = 70000 + (n_employees * 3000) + (distance / 20)
    if is_round:
        base_cost *= 1.4

    # Generate 2-4 individual costs that sum up to less than income
    total_cost = min(base_cost, max_total_cost * 0.9)
    n_costs = random.randint(2, 4)
    cost_shares = [random.uniform(0.15, 0.35) for _ in range(n_costs)]
    total_share = sum(cost_shares)
    normalized_shares = [s / total_share for s in cost_shares]

    for i in range(n_costs):
        costs.append({
            "cost_id": len(costs) + 1,
            "trip_id": trip_id,
            "cost_type": random.choice(["Fuel", "Crew", "Maintenance", "Logistics"]),
            "amount": round(total_cost * normalized_shares[i], 2)
        })

df_costs = pd.DataFrame(costs)
# df_costs.to_csv("/mnt/data/costs_generated.csv", index=False)

# # Preview costs
# import ace_tools as tools; tools.display_dataframe_to_user(name="Generated Costs", dataframe=df_costs)


clients + adress + emergency_contact

In [ ]:
# Filter clients based on their bookings and trip directions
# Also identify the two galaxies with most round-trip clients

# First, merge trip info into bookings
df_booking_trips = df_bookings.merge(df_trips, on="trip_id", how="left")
df_booking_trips = df_booking_trips.merge(df_trip_types[["triptype_id", "is_round_trip", "planet_id"]], on="triptype_id", how="left")
df_booking_trips = df_booking_trips.merge(df_planets[["planet_id", "galaxy_id"]], on="planet_id", how="left")

# Step 1: Remove clients who took at least one one-way trip
one_way_clients = df_booking_trips[df_booking_trips["is_round_trip"] == False]["client_id"].unique()
df_clients_filtered = df_clients[~df_clients["client_id"].isin(one_way_clients)]

# Step 2: For remaining clients, count round-trip bookings per galaxy
round_trip_bookings = df_booking_trips[
    (df_booking_trips["is_round_trip"] == True) &
    (df_booking_trips["client_id"].isin(df_clients_filtered["client_id"]))
]

galaxy_counts = round_trip_bookings.groupby("galaxy_id").size().sort_values(ascending=False)
top_two_galaxies = galaxy_counts.head(2).index.tolist()

# Step 3: Select clients who traveled to those galaxies
top_clients = round_trip_bookings[round_trip_bookings["galaxy_id"].isin(top_two_galaxies)]["client_id"].unique()
df_clients_top = df_clients_filtered[df_clients_filtered["client_id"].isin(top_clients)]

# Step 4: Add fake address and emergency contact info
from faker import Faker
fake = Faker()

addresses = []
emergency_contacts = []

for _, client_row in df_clients_top.iterrows():
    client_id = client_row["client_id"]
    first = client_row["first_name"].lower()
    last = client_row["last_name"].lower()
    email_username = f"{first}.{last}".replace(" ", "")
    domain = fake.free_email_domain()
    client_email = f"{email_username}@{domain}"

    # Zaktualizuj email klienta w df_clients_top
    df_clients_top.loc[df_clients_top["client_id"] == client_id, "email"] = client_email

    # Adres
    addresses.append({
        "client_id": client_id,
        "street": fake.street_address(),
        "city": fake.city(),
        "postal_code": fake.postcode(),
        "country": fake.country()
    })

    # Emergency contact (anonimowy)
    emergency_contacts.append({
        "client_id": client_id,
        "phone": fake.phone_number(),
        "email": fake.email()
    })


df_addresses = pd.DataFrame(addresses)
# Save updated data
#df_clients_top.to_csv("final_clients.csv", index=False)
#df_addresses.to_csv("final_addresses.csv", index=False)
df_emergency_contacts = pd.DataFrame(emergency_contacts)
#df_emergency_contacts.to_csv("emergency_contacts.csv", index=False)


# Show clients selected
#import ace_tools as tools; tools.display_dataframe_to_user(name="Final Clients (Round Trip & Top Galaxies)", dataframe=df_clients_top)


In [55]:
df_clients_top

,client_id,first_name,last_name,email,phone
0,1,Aaron,Thomas,aaron.thomas@yahoo.com,001-398-722-2960x3955
3,4,David,Arias,david.arias@hotmail.com,(993)876-0202x4382
14,15,David,Jones,david.jones@yahoo.com,8535127753
16,17,Steven,Allen,steven.allen@gmail.com,334-750-9174x97866
28,29,Christopher,Taylor,christopher.taylor@gmail.com,+1-338-281-9794x9239
37,38,Angel,Montoya,angel.montoya@yahoo.com,001-555-939-2673x814
40,41,Jennifer,Reese,jennifer.reese@hotmail.com,822.629.9800x790
